#### Number partitioning problem. 
Patition a set of numbers so that the sum of both partitions is as close as possible. 


##### Theory 
$ S = \{s1, s2, ... sn \} $

lets assume that in the binary allocation partition 1 becomes $ x_i = 1 $ and 0 otherwise.

$ sum_1 = \sum_{i=1}^m s_i.x_i $

$ sum_2 = total - sum_1 = \sum_{i=1}^m s_i - \sum_{i=1}^m s_i.x_i $

$ diff = sum_2 - sum_1 = \sum_{i=1}^m s_i - \sum_{i=1}^m s_i.x_i - \sum_{i=1}^m s_i.x_i = \sum_{i=1}^m s_i - 2.\sum_{i=1}^m s_i.x_i $

The goal is to minimize diff. 

$ diff^2 = ( total - 2.\sum_{i=1}^m s_i.x_i )^2 = total^2 + 4.x^tQx $

Where $ Q_{ii} = s_i(s_i - total) $ and $ Q_{ij} = s_i.s_j $

In [10]:
from qiskit_optimization import QuadraticProgram
from qiskit import BasicAer
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import QuantumInstance
from qiskit.primitives import Sampler
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit_algorithms.optimizers import COBYLA

In [11]:
S =[25, 7,13, 31, 42,17, 21,10,25, 7,13, 31, 42,17, 21,10]
C = sum(S)
Q = [[0 for _ in range(len(S))] for _ in range(len(S))]

for i in range(len(S)):
    for j in range(len(S)):
        if i == j:
            Q[i][j] = S[i]*(S[i]-C)
        else:
            Q[i][j] = S[i]*S[j]

mod = QuadraticProgram("qproblem")
[mod.binary_var() for _ in range(len(S))]
mod.minimize(quadratic=Q)
print(mod.prettyprint())

Problem name: qproblem

Minimize
  -7675*x0^2 + 350*x0*x1 + 650*x0*x10 + 1550*x0*x11 + 2100*x0*x12 + 850*x0*x13
  + 1050*x0*x14 + 500*x0*x15 + 650*x0*x2 + 1550*x0*x3 + 2100*x0*x4 + 850*x0*x5
  + 1050*x0*x6 + 500*x0*x7 + 1250*x0*x8 + 350*x0*x9 - 2275*x1^2 + 182*x1*x10
  + 434*x1*x11 + 588*x1*x12 + 238*x1*x13 + 294*x1*x14 + 140*x1*x15 + 182*x1*x2
  + 434*x1*x3 + 588*x1*x4 + 238*x1*x5 + 294*x1*x6 + 140*x1*x7 + 350*x1*x8
  + 98*x1*x9 - 4147*x10^2 + 806*x10*x11 + 1092*x10*x12 + 442*x10*x13
  + 546*x10*x14 + 260*x10*x15 - 9331*x11^2 + 2604*x11*x12 + 1054*x11*x13
  + 1302*x11*x14 + 620*x11*x15 - 12180*x12^2 + 1428*x12*x13 + 1764*x12*x14
  + 840*x12*x15 - 5355*x13^2 + 714*x13*x14 + 340*x13*x15 - 6531*x14^2
  + 420*x14*x15 - 3220*x15^2 + 338*x2*x10 + 806*x2*x11 + 1092*x2*x12
  + 442*x2*x13 + 546*x2*x14 + 260*x2*x15 - 4147*x2^2 + 806*x2*x3 + 1092*x2*x4
  + 442*x2*x5 + 546*x2*x6 + 260*x2*x7 + 650*x2*x8 + 182*x2*x9 + 806*x3*x10
  + 1922*x3*x11 + 2604*x3*x12 + 1054*x3*x13 + 1302*x3*x14 + 620*x3*x15

In [12]:
quantum_instance = Sampler()
qaoa_mes = QAOA(quantum_instance, COBYLA(), reps=2)
qaoa = MinimumEigenOptimizer(qaoa_mes)

qaoa_result = qaoa.solve(mod)
print([qaoa_result.x], [qaoa_result.fval])

p1 = 0
p2 = 0

for i in range(len(S)):
    if qaoa_result.x[i] == 1:
        p1 += S[i]
    else:
        p2 += S[i]

print (p1, p2)


[array([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])] [-27556.0]
166 166
